<a href="https://colab.research.google.com/github/LinZhihao-723/aps360_team66/blob/master/APS360_team_66.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
def get_Train_Val_loader(batch_size = 1, 
                    path_train = '/content/gdrive/MyDrive/Project_Data/training', 
                    path_val = '/content/gdrive/MyDrive/Project_Data/validation',
                    path_test = '/content/gdrive/MyDrive/Project_Data/testing',
                    seed = 1000):
    transform = transforms.Compose(
        [transforms.Resize(48*4),
         transforms.CenterCrop(48*4), transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
    dataset_train = torchvision.datasets.ImageFolder(root = path_train, transform = transform)
    dataset_val = torchvision.datasets.ImageFolder(root = path_val, transform = transform)
    dataset_test = torchvision.datasets.ImageFolder(root = path_test, transform = transform)
    indices_train = list(range(len(dataset_train)))
    indices_val = list(range(len(dataset_val)))
    indices_test = list(range(len(dataset_test)))

    np.random.seed(seed) # Fixed numpy random seed for reproducible shuffling
    np.random.shuffle(indices_train)
    np.random.shuffle(indices_val)
    
    train_sampler = SubsetRandomSampler(indices_train)
    train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size,
                                               num_workers=1, sampler=train_sampler)
  
    val_sampler = SubsetRandomSampler(indices_val)
    val_loader = torch.utils.data.DataLoader(dataset_val, batch_size=batch_size,
                                              num_workers=1, sampler=val_sampler)
    test_sampler = SubsetRandomSampler(indices_test)
    test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size,
                                              num_workers=1, sampler=test_sampler)
    print('train set size: ', len(indices_train))
    print('validation set size: ', len(indices_val))
    print('test set size: ', len(indices_test))
    return train_loader, val_loader, test_loader

In [ ]:
a, b, c = get_Train_Val_loader(batch_size=1)

train set size:  4864
validation set size:  1159
test set size:  1223


In [ ]:
import torchvision.models
alexnet = torchvision.models.alexnet(pretrained=True)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth


In [ ]:
####DO NOT RUN ANYMORE
def load_alexnet(train_loader, val_loader, test_loader,
                 path_alexnet = '/content/gdrive/MyDrive/Project_Data/Data_alexnet'):  
    expression_class = ['positive', 'negative', 'neutral']
    i,j,k = 0,0,0;
    for img, type_expression in train_loader:
      #print(img.size(), expression_class[type_expression])
      features = alexnet.features(img)
      features = torch.from_numpy(features.detach().numpy())
      #print(type_expression)
      #print(img)
      torch.save(features.squeeze(0), path_alexnet + '/training/' + str(expression_class[type_expression]) + '/' + str(i) + '.tensor')
      i = i + 1

    for img, type_expression in val_loader:
      features = alexnet.features(img)
      features = torch.from_numpy(features.detach().numpy())
      torch.save(features.squeeze(0), path_alexnet + '/validation/' + str(expression_class[type_expression]) + '/' + str(j) + '.tensor')
      j = j + 1
    for img, type_expression in test_loader:
      features = alexnet.features(img)
      features = torch.from_numpy(features.detach().numpy())
      torch.save(features.squeeze(0), path_alexnet + '/testing/' + str(expression_class[type_expression]) + '/' + str(k) + '.tensor')
      k = k + 1

In [ ]:
# don't run don't run don't run don't run!!!!!!!!!!!!!!!!!!!!
# don't run don't run don't run don't run!!!!!!!!!!!!!!!!!!!!
# don't run don't run don't run don't run!!!!!!!!!!!!!!!!!!!!
#load_alexnet(train_loader = a, val_loader = b, test_loader = c)

In [ ]:
def get_alexnet_loader(batch_size = 10,
                    path_alex_train = '/content/gdrive/MyDrive/Project_Data/Data_alexnet/training', 
                    path_alex_val = '/content/gdrive/MyDrive/Project_Data/Data_alexnet/validation',
                    path_alex_test = '/content/gdrive/MyDrive/Project_Data/Data_alexnet/testing',
                    seed = 1000):
    # Load CIFAR10 training data
    alex_data_set_train = torchvision.datasets.DatasetFolder(root = path_alex_train, 
                                                       loader=torch.load, extensions=('.tensor'))
    alex_data_set_val = torchvision.datasets.DatasetFolder(root = path_alex_val, 
                                                       loader=torch.load, extensions=('.tensor'))
    alex_data_set_test = torchvision.datasets.DatasetFolder(root = path_alex_test, 
                                                       loader=torch.load, extensions=('.tensor'))
    all_indices_train = list(range(len(alex_data_set_train)));
    all_indices_val = list(range(len(alex_data_set_val)));
    all_indices_test = list(range(len(alex_data_set_test)));

    np.random.seed(seed) # Fixed numpy random seed for reproducible shuffling
    np.random.shuffle(all_indices_train)
    np.random.shuffle(all_indices_val)
    np.random.shuffle(all_indices_test)
    
    train_sampler = SubsetRandomSampler(all_indices_train)
    train_loader = torch.utils.data.DataLoader(alex_data_set_train, batch_size=batch_size,
                                               num_workers=1, sampler=train_sampler)
    val_sampler = SubsetRandomSampler(all_indices_val)
    val_loader = torch.utils.data.DataLoader(alex_data_set_val, batch_size=batch_size,
                                              num_workers=1, sampler=val_sampler)
    test_sampler = SubsetRandomSampler(all_indices_test)
    test_loader = torch.utils.data.DataLoader(alex_data_set_test, batch_size=batch_size,
                                              num_workers=1, sampler=test_sampler)
    
    print('train set size: ', len(all_indices_train))
    print('validation set size: ', len(all_indices_val))
    print('test set size: ', len(all_indices_test))
    return train_loader, val_loader, test_loader

In [ ]:
get_alexnet_loader(batch_size = 10)

train set size:  4864
validation set size:  1159
test set size:  1223


(<torch.utils.data.dataloader.DataLoader at 0x7f50b15507d0>,
 <torch.utils.data.dataloader.DataLoader at 0x7f505fcbe210>)

**Model and Training**

In [ ]:
class CNN(nn.Module):
  def __init__(self):
        super(CNN, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(256, 128, 1),
            nn.ReLU(),
            nn.Conv2d(128, 64, 3),
            nn.ReLU(),
            nn.Conv2d(64, 32, 1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(32 * 3 * 3, 500),
            nn.Dropout(0.5), #20% probability 
            nn.ReLU(),
            nn.Linear(500, 300),
            nn.Dropout(0.5), #50% probability 
            # nn.Dropout(0.2), #50% probability 
            nn.ReLU(),
            nn.Linear(300, 3)
        )

  def forward(self, x):
        x = self.layers(x)
        return x

class Benchmark_NN(nn.Module):
  def __init__(self):
        super(Benchmark_NN, self).__init__()
        self.fc1 = nn.Linear(256 * 5 * 5, 500)
        self.fc2 = nn.Linear(500, 300)
        self.fc3 = nn.Linear(300, 3)

  def forward(self, x):
        x = x.view(-1, 256 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# This function is used to evaluate the model, by passing a validation set or a test set.
def evaluate(net, loader, batch_size):
  criterion = nn.CrossEntropyLoss()
  num_error = 0
  num_total = 0
  total_loss = 0.0
  for i, data in enumerate(loader, 0):
    inputs, labels = data
    outputs = net(inputs)
    loss = criterion(outputs, labels)   
    total_loss += loss.item() * inputs.size(0)
    for index, target in enumerate(outputs):
      num_total += 1
      if torch.argmax(target) != labels[index]:
        num_error += 1
  return total_loss/num_total, num_error/num_total

def final_evaluate(net, loader):
  num_label_total = [0, 0, 0]
  num_label_predict = [0, 0, 0]
  num_label_correct = [0, 0, 0]
  num_label_miss = [0, 0, 0]
  num_label_confused = [0, 0, 0]
  overall_num = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
  criterion = nn.CrossEntropyLoss()
  num_error = 0
  num_total = 0
  total_loss = 0.0
  for i, data in enumerate(loader, 0):
    inputs, labels = data
    outputs = net(inputs)
    loss = criterion(outputs, labels)   
    total_loss += loss.item() * inputs.size(0)
    for index, target in enumerate(outputs):
      num_total += 1
      num_label_total[labels[index]] += 1
      num_label_predict[torch.argmax(target)] += 1
      overall_num[labels[index]][torch.argmax(target)] += 1
      if torch.argmax(target) != labels[index]:
        num_error += 1
        num_label_miss[labels[index]] += 1
        num_label_confused[torch.argmax(target)] += 1
      else:
        num_label_correct[torch.argmax(target)] += 1
  print("Total: ", num_label_total)
  print("Predict: ", num_label_predict)
  print("Correct: ", num_label_correct)
  print("Miss: ", num_label_miss)
  print("Confused: ", num_label_confused)
  print("Overall: ", overall_num)
  return total_loss/num_total, num_error/num_total

# Loaders should be processed before calling this function
def training_process(net, batch_size, learning_rate, num_epochs, train_loader, validation_loader):
  torch.manual_seed(3190)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

  print("Start training")
  start_time = time.time()

  train_err_list = np.zeros(num_epochs)
  train_loss_list = np.zeros(num_epochs)
  val_err_list = np.zeros(num_epochs)
  val_loss_list = np.zeros(num_epochs)

  for epoch in range(num_epochs):
    train_loss = 0.0
    num_train_total = 0
    num_train_error = 0

    # training:
    for i, data in enumerate(train_loader, 0):
      inputs, labels = data
      optimizer.zero_grad()
      outputs = net(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      train_loss += loss.item() * inputs.size(0)
      for index, target in enumerate(outputs):
        num_train_total += 1
        if torch.argmax(target) != labels[index]:
          num_train_error += 1
    train_loss_list[epoch] = train_loss / num_train_total
    train_err_list[epoch] = num_train_error / num_train_total
    # validation:
    val_loss_list[epoch], val_err_list[epoch] = evaluate(net, validation_loader, batch_size)
    
    print(("Epoch {}: Train err: {}, Train loss: {} |"+
               "Validation err: {}, Validation loss: {}").format(
                   epoch + 1,
                   train_err_list[epoch],
                   train_loss_list[epoch],
                   val_err_list[epoch],
                   val_loss_list[epoch]))
    
  print('Finished Training')
  end_time = time.time()
  elapsed_time = end_time - start_time
  print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

  # Plot error
  plt.title("Train vs Validation Error")
  n = len(train_err_list) # number of epochs
  plt.plot(range(1,n+1), train_err_list, label="Train")
  plt.plot(range(1,n+1), val_err_list, label="Validation")
  plt.xlabel("Epoch")
  plt.ylabel("Error")
  plt.legend(loc='best')
  plt.show()

  # Plot loss
  plt.title("Train vs Validation Loss")
  plt.plot(range(1,n+1), train_loss_list, label="Train")
  plt.plot(range(1,n+1), val_loss_list, label="Validation")
  plt.xlabel("Epoch")
  plt.ylabel("Loss")
  plt.legend(loc='best')
  plt.show()